In [ ]:
#%%

import os
import torch
from data_processing import dataset_preprocessing, data_with_mask
from sph_functions import interpolation_to_grid, hemisphere_to_spherical, spherical_to_hemisphere
from plots_pavi import plot_sphere

from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from scipy.stats.mstats import pearsonr
import numpy as np

from model import Autoencoder, initialize_model, train_autoencoder, extract_latent_space

DIR = r'C:\Github\pavi_data'

path_file   = os.path.join(DIR, '12_mean_sample_post.pt')
path_jobs   = os.path.join(DIR, 'jobs.json')
path_scores = os.path.join(DIR, 'scores_camcan.csv')
n_job = 44


net_number = None
sh_orders = 4

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


X, y, seed = dataset_preprocessing(DEVICE, path_file, path_jobs, path_scores)

vertices_left, vertices_right, network_left, network_right = data_with_mask(X, net_number)


fancy_categories = [
        'Benton faces',
        'Fluid Intelligence',
        'Emotion expression recognition',
        'Famous faces',
        'Hotel task',
        'Picture priming',
        'Proverb comprehension',
        'Sentence comprehension (unacceptable error)',
        'Sentence comprehension (reaction time)',
        'Visual short term memory (mean)',
        'Visual short term memory (precision)',
        'Visual short term memory (doubt)',
        'Visual short term memory (MSE)',
    ]
n_scores = len(fancy_categories)

pixdim[1,2,3] should be non-zero; setting 0 dims to 1


In [ ]:
#%%

mesh_theta, sphere_src_left, sphere_src_right, sphere_dst = interpolation_to_grid(vertices_left, vertices_right, sh_orders)
sph_data_left  = hemisphere_to_spherical(network_left, sphere_src_left, sphere_dst, sh_orders)
sph_data_right = hemisphere_to_spherical(network_right, sphere_src_right, sphere_dst, sh_orders)

print(sph_data_left.shape)
print(sph_data_right.shape)

sph_data_left = sph_data_left.reshape(-1,mesh_theta.shape[0], mesh_theta.shape[1],network_left.shape[-1])
sph_data_right = sph_data_right.reshape(-1,mesh_theta.shape[0], mesh_theta.shape[1],network_right.shape[-1])

if isinstance(net_number, int):
    plot_sphere(sph_data_left, net_number)
    plot_sphere(sph_data_right, net_number)

print("New shape:")
print(sph_data_left.shape)
print(sph_data_right.shape)

sh_order=4
(218, 32258, 17)
(218, 32258, 17)
New shape:
(218, 127, 254, 17)
(218, 127, 254, 17)


In [ ]:
#%%

import os
import torch
from data_processing import dataset_preprocessing, data_with_mask
from sph_functions import interpolation_to_grid, hemisphere_to_spherical, spherical_to_hemisphere
from plots_pavi import plot_sphere

from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from scipy.stats.mstats import pearsonr
import numpy as np

from model import Autoencoder, initialize_model, train_autoencoder, extract_latent_space

DIR = r'C:\Github\pavi_data'

path_file   = os.path.join(DIR, '12_mean_sample_post.pt')
path_jobs   = os.path.join(DIR, 'jobs.json')
path_scores = os.path.join(DIR, 'scores_camcan.csv')
n_job = 44


net_number = None
sh_orders = 16

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


X, y, seed = dataset_preprocessing(DEVICE, path_file, path_jobs, path_scores)

vertices_left, vertices_right, network_left, network_right = data_with_mask(X, net_number)


fancy_categories = [
        'Benton faces',
        'Fluid Intelligence',
        'Emotion expression recognition',
        'Famous faces',
        'Hotel task',
        'Picture priming',
        'Proverb comprehension',
        'Sentence comprehension (unacceptable error)',
        'Sentence comprehension (reaction time)',
        'Visual short term memory (mean)',
        'Visual short term memory (precision)',
        'Visual short term memory (doubt)',
        'Visual short term memory (MSE)',
    ]
n_scores = len(fancy_categories)

In [ ]:
#%%

mesh_theta, sphere_src_left, sphere_src_right, sphere_dst = interpolation_to_grid(vertices_left, vertices_right, sh_orders)
sph_data_left  = hemisphere_to_spherical(network_left, sphere_src_left, sphere_dst, sh_orders)
sph_data_right = hemisphere_to_spherical(network_right, sphere_src_right, sphere_dst, sh_orders)

print(sph_data_left.shape)
print(sph_data_right.shape)

sph_data_left = sph_data_left.reshape(-1,mesh_theta.shape[0], mesh_theta.shape[1],network_left.shape[-1])
sph_data_right = sph_data_right.reshape(-1,mesh_theta.shape[0], mesh_theta.shape[1],network_right.shape[-1])

if isinstance(net_number, int):
    plot_sphere(sph_data_left, net_number)
    plot_sphere(sph_data_right, net_number)

print("New shape:")
print(sph_data_left.shape)
print(sph_data_right.shape)

sh_order=16
(218, 32258, 17)
(218, 32258, 17)
New shape:
(218, 127, 254, 17)
(218, 127, 254, 17)


In [ ]:
#%%

autoencoders_left  = [Autoencoder(127, 254, kernel=30).to(DEVICE) for _ in range(sph_data_left.shape[-1])]
autoencoders_right = [Autoencoder(127, 254, kernel=30).to(DEVICE) for _ in range(sph_data_left.shape[-1])]

for ae_l, ae_r in zip(autoencoders_left, autoencoders_right):
    initialize_model(ae_l)
    initialize_model(ae_r)

c:\Users\joaqu\anaconda3\envs\data_science\Lib\site-packages\torch\functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3588.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [ ]:
#%%

# Joint split for the left and right hemisphere networks and the corresponding labels (y).

# Step 1: Perform a joint split based on the indices
# Generate the same train/test split indices for both hemispheres
train_indices, test_indices = train_test_split(np.arange(sph_data_left.shape[0]), test_size=0.2, random_state=42)

# Step 2: Split the data using these indices
train_data_left = sph_data_left[train_indices]
test_data_left = sph_data_left[test_indices]
train_data_right = sph_data_left[train_indices]
test_data_right = sph_data_left[test_indices]

# Step 3: Split the target variable y using the same indices
y_train = y[train_indices]
y_test = y[test_indices]

In [ ]:
#%%

train_latent_space_left = np.zeros((train_data_left.shape[0], 32, 64, train_data_left.shape[-1]), dtype=np.float32)
train_latent_space_right = np.zeros((train_data_right.shape[0], 32, 64, train_data_left.shape[-1]), dtype=np.float32)


for i in range (train_data_left.shape[-1]):
    print (f"Network number {1+i}")
    train_autoencoder(autoencoders_left[i],  train_data_left[:,:,:,i], DEVICE, num_iterations=100)
    train_autoencoder(autoencoders_right[i], train_data_right[:,:,:,i], DEVICE, num_iterations=100)

    lsp_left  = extract_latent_space(autoencoders_left[i], train_data_left[:,:,:,i], DEVICE)
    lsp_right = extract_latent_space(autoencoders_right[i],train_data_right[:,:,:,i], DEVICE)

    train_latent_space_left[:, :, :, i]  = lsp_left
    train_latent_space_right[:, :, :, i] = lsp_right

Network number 1
Iteration 0, Loss: 0.00011646038910839707
Iteration 50, Loss: 8.717192395124584e-05
Iteration 0, Loss: 0.00011646038910839707
Iteration 50, Loss: 8.717192395124584e-05
Network number 2
Iteration 0, Loss: 3.821830250672065e-05
Iteration 50, Loss: 1.114604947360931e-05
Iteration 0, Loss: 3.821830250672065e-05
Iteration 50, Loss: 1.114604947360931e-05
Network number 3
Iteration 0, Loss: 8.042909030336887e-05
Iteration 50, Loss: 0.018725931644439697
Iteration 0, Loss: 8.042909030336887e-05
Iteration 50, Loss: 0.018725931644439697
Network number 4
Iteration 0, Loss: 0.00013321891310624778
Iteration 50, Loss: 0.03227923437952995
Iteration 0, Loss: 0.00013321891310624778
Iteration 50, Loss: 0.03227923437952995
Network number 5
Iteration 0, Loss: 1.7260834283661097e-05
Iteration 50, Loss: 5.736729690397624e-06
Iteration 0, Loss: 1.7260834283661097e-05
Iteration 50, Loss: 5.736729690397624e-06
Network number 6
Iteration 0, Loss: 4.007687311968766e-05
Iteration 50, Loss: 0.00130

In [ ]:
#%%

tr_lsp_lft = train_latent_space_left.reshape(train_latent_space_left.shape[0], -1)
tr_lsp_rght = train_latent_space_right.reshape(train_latent_space_right.shape[0], -1)

tr_lsp = np.concatenate ((tr_lsp_lft, tr_lsp_rght), axis=1)

In [ ]:
# %%

test_latent_space_left = np.zeros((test_data_left.shape[0], 32, 64,   test_data_left.shape[-1]), dtype=np.float32)
test_latent_space_right = np.zeros((test_data_right.shape[0], 32, 64, test_data_right.shape[-1]), dtype=np.float32)

for i in range (test_data_left.shape[-1]):
    print (f"Network number {1+i}")
    lsp_left  = extract_latent_space(autoencoders_left[i],  test_data_left[:,:,:,i], DEVICE)
    lsp_right = extract_latent_space(autoencoders_right[i], test_data_right[:,:,:,i], DEVICE)

    test_latent_space_left[:, :, :, i]  = lsp_left
    test_latent_space_right[:, :, :, i] = lsp_right

Network number 1
Network number 2
Network number 3
Network number 4
Network number 5
Network number 6
Network number 7
Network number 8
Network number 9
Network number 10
Network number 11
Network number 12
Network number 13
Network number 14
Network number 15
Network number 16
Network number 17


In [ ]:
#%%

tst_lsp_lft = test_latent_space_left.reshape(test_latent_space_left.shape[0], -1)
tst_lsp_rght = test_latent_space_right.reshape(test_latent_space_right.shape[0], -1)

tst_lsp = np.concatenate((tst_lsp_lft, tst_lsp_rght), axis=1)

In [ ]:
# %%

# Regresion final.
model = Ridge()
model.fit(tr_lsp, y_train)
y_pred = model.predict(tst_lsp)
scores_ = np.full((n_scores,), np.nan)
for c in range(n_scores):
    r, _ = pearsonr(y_pred[:, c], y_test[:, c])
    scores_[c] = r

In [ ]:
#%%

scores_.mean()

0.24433748920121828

In [ ]:
#%%

import os
import torch
from data_processing import dataset_preprocessing, data_with_mask
from sph_functions import interpolation_to_grid, hemisphere_to_spherical, spherical_to_hemisphere
from plots_pavi import plot_sphere

from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from scipy.stats.mstats import pearsonr
import numpy as np

from model import Autoencoder, initialize_model, train_autoencoder, extract_latent_space

DIR = r'C:\Github\pavi_data'

path_file   = os.path.join(DIR, '12_mean_sample_post.pt')
path_jobs   = os.path.join(DIR, 'jobs.json')
path_scores = os.path.join(DIR, 'scores_camcan.csv')
n_job = 44


net_number = None
sh_orders = 32

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


X, y, seed = dataset_preprocessing(DEVICE, path_file, path_jobs, path_scores)

vertices_left, vertices_right, network_left, network_right = data_with_mask(X, net_number)


fancy_categories = [
        'Benton faces',
        'Fluid Intelligence',
        'Emotion expression recognition',
        'Famous faces',
        'Hotel task',
        'Picture priming',
        'Proverb comprehension',
        'Sentence comprehension (unacceptable error)',
        'Sentence comprehension (reaction time)',
        'Visual short term memory (mean)',
        'Visual short term memory (precision)',
        'Visual short term memory (doubt)',
        'Visual short term memory (MSE)',
    ]
n_scores = len(fancy_categories)

In [ ]:
#%%

mesh_theta, sphere_src_left, sphere_src_right, sphere_dst = interpolation_to_grid(vertices_left, vertices_right, sh_orders)
sph_data_left  = hemisphere_to_spherical(network_left, sphere_src_left, sphere_dst, sh_orders)
sph_data_right = hemisphere_to_spherical(network_right, sphere_src_right, sphere_dst, sh_orders)

print(sph_data_left.shape)
print(sph_data_right.shape)

sph_data_left = sph_data_left.reshape(-1,mesh_theta.shape[0], mesh_theta.shape[1],network_left.shape[-1])
sph_data_right = sph_data_right.reshape(-1,mesh_theta.shape[0], mesh_theta.shape[1],network_right.shape[-1])

if isinstance(net_number, int):
    plot_sphere(sph_data_left, net_number)
    plot_sphere(sph_data_right, net_number)

print("New shape:")
print(sph_data_left.shape)
print(sph_data_right.shape)

sh_order=32
(218, 32258, 17)
(218, 32258, 17)
New shape:
(218, 127, 254, 17)
(218, 127, 254, 17)


In [ ]:
#%%

autoencoders_left  = [Autoencoder(127, 254, kernel=30).to(DEVICE) for _ in range(sph_data_left.shape[-1])]
autoencoders_right = [Autoencoder(127, 254, kernel=30).to(DEVICE) for _ in range(sph_data_left.shape[-1])]

for ae_l, ae_r in zip(autoencoders_left, autoencoders_right):
    initialize_model(ae_l)
    initialize_model(ae_r)

In [ ]:
#%%

# Joint split for the left and right hemisphere networks and the corresponding labels (y).

# Step 1: Perform a joint split based on the indices
# Generate the same train/test split indices for both hemispheres
train_indices, test_indices = train_test_split(np.arange(sph_data_left.shape[0]), test_size=0.2, random_state=42)

# Step 2: Split the data using these indices
train_data_left = sph_data_left[train_indices]
test_data_left = sph_data_left[test_indices]
train_data_right = sph_data_left[train_indices]
test_data_right = sph_data_left[test_indices]

# Step 3: Split the target variable y using the same indices
y_train = y[train_indices]
y_test = y[test_indices]

In [ ]:
#%%

train_latent_space_left = np.zeros((train_data_left.shape[0], 32, 64, train_data_left.shape[-1]), dtype=np.float32)
train_latent_space_right = np.zeros((train_data_right.shape[0], 32, 64, train_data_left.shape[-1]), dtype=np.float32)


for i in range (train_data_left.shape[-1]):
    print (f"Network number {1+i}")
    train_autoencoder(autoencoders_left[i],  train_data_left[:,:,:,i], DEVICE, num_iterations=100)
    train_autoencoder(autoencoders_right[i], train_data_right[:,:,:,i], DEVICE, num_iterations=100)

    lsp_left  = extract_latent_space(autoencoders_left[i], train_data_left[:,:,:,i], DEVICE)
    lsp_right = extract_latent_space(autoencoders_right[i],train_data_right[:,:,:,i], DEVICE)

    train_latent_space_left[:, :, :, i]  = lsp_left
    train_latent_space_right[:, :, :, i] = lsp_right

Network number 1
Iteration 0, Loss: 0.007351523265242577
Iteration 50, Loss: 0.00703839398920536
Iteration 0, Loss: 0.007351523265242577
Iteration 50, Loss: 0.00703839398920536
Network number 2
Iteration 0, Loss: 0.0031178086064755917
Iteration 50, Loss: 0.003041125601157546
Iteration 0, Loss: 0.0031178086064755917
Iteration 50, Loss: 0.003041125601157546
Network number 3
Iteration 0, Loss: 0.006489838473498821
Iteration 50, Loss: 0.023547230288386345
Iteration 0, Loss: 0.006489838473498821
Iteration 50, Loss: 0.023547230288386345
Network number 4
Iteration 0, Loss: 0.006615982856601477
Iteration 50, Loss: 0.0902128517627716
Iteration 0, Loss: 0.006615982856601477
Iteration 50, Loss: 0.0902128517627716
Network number 5
Iteration 0, Loss: 0.003936176188290119
Iteration 50, Loss: 0.003816701238974929
Iteration 0, Loss: 0.003936176188290119
Iteration 50, Loss: 0.003816701238974929
Network number 6
Iteration 0, Loss: 0.005872378591448069
Iteration 50, Loss: 0.007325686048716307
Iteration 0

In [ ]:
#%%

tr_lsp_lft = train_latent_space_left.reshape(train_latent_space_left.shape[0], -1)
tr_lsp_rght = train_latent_space_right.reshape(train_latent_space_right.shape[0], -1)

tr_lsp = np.concatenate ((tr_lsp_lft, tr_lsp_rght), axis=1)

In [ ]:
# %%

test_latent_space_left = np.zeros((test_data_left.shape[0], 32, 64,   test_data_left.shape[-1]), dtype=np.float32)
test_latent_space_right = np.zeros((test_data_right.shape[0], 32, 64, test_data_right.shape[-1]), dtype=np.float32)

for i in range (test_data_left.shape[-1]):
    print (f"Network number {1+i}")
    lsp_left  = extract_latent_space(autoencoders_left[i],  test_data_left[:,:,:,i], DEVICE)
    lsp_right = extract_latent_space(autoencoders_right[i], test_data_right[:,:,:,i], DEVICE)

    test_latent_space_left[:, :, :, i]  = lsp_left
    test_latent_space_right[:, :, :, i] = lsp_right

Network number 1
Network number 2
Network number 3
Network number 4
Network number 5
Network number 6
Network number 7
Network number 8
Network number 9
Network number 10
Network number 11
Network number 12
Network number 13
Network number 14
Network number 15
Network number 16
Network number 17


In [ ]:
#%%

tst_lsp_lft = test_latent_space_left.reshape(test_latent_space_left.shape[0], -1)
tst_lsp_rght = test_latent_space_right.reshape(test_latent_space_right.shape[0], -1)

tst_lsp = np.concatenate((tst_lsp_lft, tst_lsp_rght), axis=1)

In [ ]:
# %%

# Regresion final.
model = Ridge()
model.fit(tr_lsp, y_train)
y_pred = model.predict(tst_lsp)
scores_ = np.full((n_scores,), np.nan)
for c in range(n_scores):
    r, _ = pearsonr(y_pred[:, c], y_test[:, c])
    scores_[c] = r

In [ ]:
#%%

scores_.mean()

0.2486250689781005

In [ ]:
#%%

import os
import torch
from data_processing import dataset_preprocessing, data_with_mask
from sph_functions import interpolation_to_grid, hemisphere_to_spherical
from plots_pavi import plot_sphere

from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from scipy.stats.mstats import pearsonr
import numpy as np

from model import Autoencoder, initialize_model, train_autoencoder, extract_latent_space

DIR = r'C:\Github\pavi_data'

path_file   = os.path.join(DIR, '12_mean_sample_post.pt')
path_jobs   = os.path.join(DIR, 'jobs.json')
path_scores = os.path.join(DIR, 'scores_camcan.csv')
n_job = 44


net_number = None
sh_orders = 80

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


X, y, seed = dataset_preprocessing(DEVICE, path_file, path_jobs, path_scores)

vertices_left, vertices_right, network_left, network_right = data_with_mask(X, net_number)


fancy_categories = [
        'Benton faces',
        'Fluid Intelligence',
        'Emotion expression recognition',
        'Famous faces',
        'Hotel task',
        'Picture priming',
        'Proverb comprehension',
        'Sentence comprehension (unacceptable error)',
        'Sentence comprehension (reaction time)',
        'Visual short term memory (mean)',
        'Visual short term memory (precision)',
        'Visual short term memory (doubt)',
        'Visual short term memory (MSE)',
    ]
n_scores = len(fancy_categories)

In [ ]:
#%%

mesh_theta, sphere_src_left, sphere_src_right, sphere_dst = interpolation_to_grid(vertices_left, vertices_right, sh_orders)
sph_data_left  = hemisphere_to_spherical(network_left, sphere_src_left, sphere_dst, sh_orders)
sph_data_right = hemisphere_to_spherical(network_right, sphere_src_right, sphere_dst, sh_orders)

print(sph_data_left.shape)
print(sph_data_right.shape)

sph_data_left = sph_data_left.reshape(-1,mesh_theta.shape[0], mesh_theta.shape[1],network_left.shape[-1])
sph_data_right = sph_data_right.reshape(-1,mesh_theta.shape[0], mesh_theta.shape[1],network_right.shape[-1])

if isinstance(net_number, int):
    plot_sphere(sph_data_left, net_number)
    plot_sphere(sph_data_right, net_number)

print("New shape:")
print(sph_data_left.shape)
print(sph_data_right.shape)

sh_order=80
(218, 32258, 17)
(218, 32258, 17)
New shape:
(218, 127, 254, 17)
(218, 127, 254, 17)


In [ ]:
#%%

autoencoders_left  = [Autoencoder(127, 254, kernel=30).to(DEVICE) for _ in range(sph_data_left.shape[-1])]
autoencoders_right = [Autoencoder(127, 254, kernel=30).to(DEVICE) for _ in range(sph_data_left.shape[-1])]

for ae_l, ae_r in zip(autoencoders_left, autoencoders_right):
    initialize_model(ae_l)
    initialize_model(ae_r)

In [ ]:
#%%

# Joint split for the left and right hemisphere networks and the corresponding labels (y).

# Step 1: Perform a joint split based on the indices
# Generate the same train/test split indices for both hemispheres
train_indices, test_indices = train_test_split(np.arange(sph_data_left.shape[0]), test_size=0.2, random_state=42)

# Step 2: Split the data using these indices
train_data_left = sph_data_left[train_indices]
test_data_left = sph_data_left[test_indices]
train_data_right = sph_data_left[train_indices]
test_data_right = sph_data_left[test_indices]

# Step 3: Split the target variable y using the same indices
y_train = y[train_indices]
y_test = y[test_indices]

In [ ]:
#%%

train_latent_space_left = np.zeros((train_data_left.shape[0], 32, 64, train_data_left.shape[-1]), dtype=np.float32)
train_latent_space_right = np.zeros((train_data_right.shape[0], 32, 64, train_data_left.shape[-1]), dtype=np.float32)


for i in range (train_data_left.shape[-1]):
    print (f"Network number {1+i}")
    train_autoencoder(autoencoders_left[i],  train_data_left[:,:,:,i], DEVICE, num_iterations=100)
    train_autoencoder(autoencoders_right[i], train_data_right[:,:,:,i], DEVICE, num_iterations=100)

    lsp_left  = extract_latent_space(autoencoders_left[i], train_data_left[:,:,:,i], DEVICE)
    lsp_right = extract_latent_space(autoencoders_right[i],train_data_right[:,:,:,i], DEVICE)

    train_latent_space_left[:, :, :, i]  = lsp_left
    train_latent_space_right[:, :, :, i] = lsp_right

Network number 1
Iteration 0, Loss: 0.10739010572433472
Iteration 50, Loss: 0.10279100388288498
Iteration 0, Loss: 0.10739010572433472
Iteration 50, Loss: 0.10279100388288498
Network number 2
Iteration 0, Loss: 0.055353179574012756
Iteration 50, Loss: 0.05329129844903946
Iteration 0, Loss: 0.055353179574012756
Iteration 50, Loss: 0.05329129844903946
Network number 3
Iteration 0, Loss: 0.10854646563529968
Iteration 50, Loss: 0.10400619357824326
Iteration 0, Loss: 0.10854646563529968
Iteration 50, Loss: 0.10400619357824326
Network number 4
Iteration 0, Loss: 0.10516277700662613
Iteration 50, Loss: 0.11232051253318787
Iteration 0, Loss: 0.10516277700662613
Iteration 50, Loss: 0.11232051253318787
Network number 5
Iteration 0, Loss: 0.07743297517299652
Iteration 50, Loss: 0.07389683276414871
Iteration 0, Loss: 0.07743297517299652
Iteration 50, Loss: 0.07389683276414871
Network number 6
Iteration 0, Loss: 0.10648210346698761
Iteration 50, Loss: 0.1010470762848854
Iteration 0, Loss: 0.1064821

In [ ]:
#%%

tr_lsp_lft = train_latent_space_left.reshape(train_latent_space_left.shape[0], -1)
tr_lsp_rght = train_latent_space_right.reshape(train_latent_space_right.shape[0], -1)

tr_lsp = np.concatenate ((tr_lsp_lft, tr_lsp_rght), axis=1)

In [ ]:
# %%

test_latent_space_left = np.zeros((test_data_left.shape[0], 32, 64,   test_data_left.shape[-1]), dtype=np.float32)
test_latent_space_right = np.zeros((test_data_right.shape[0], 32, 64, test_data_right.shape[-1]), dtype=np.float32)

for i in range (test_data_left.shape[-1]):
    print (f"Network number {1+i}")
    lsp_left  = extract_latent_space(autoencoders_left[i],  test_data_left[:,:,:,i], DEVICE)
    lsp_right = extract_latent_space(autoencoders_right[i], test_data_right[:,:,:,i], DEVICE)

    test_latent_space_left[:, :, :, i]  = lsp_left
    test_latent_space_right[:, :, :, i] = lsp_right

Network number 1
Network number 2
Network number 3
Network number 4
Network number 5
Network number 6
Network number 7
Network number 8
Network number 9
Network number 10
Network number 11
Network number 12
Network number 13
Network number 14
Network number 15
Network number 16
Network number 17


In [ ]:
#%%

tst_lsp_lft = test_latent_space_left.reshape(test_latent_space_left.shape[0], -1)
tst_lsp_rght = test_latent_space_right.reshape(test_latent_space_right.shape[0], -1)

tst_lsp = np.concatenate((tst_lsp_lft, tst_lsp_rght), axis=1)

In [ ]:
# %%

# Regresion final.
model = Ridge()
model.fit(tr_lsp, y_train)
y_pred = model.predict(tst_lsp)
scores_ = np.full((n_scores,), np.nan)
for c in range(n_scores):
    r, _ = pearsonr(y_pred[:, c], y_test[:, c])
    scores_[c] = r

In [ ]:
#%%

scores_.mean()

0.24300207164287205